In [ ]:
curl https://sh.rustup.rs -sSf | sh

In [ ]:
import sys
!{sys.executable} -m pip install transformers==3.0.0

In [ ]:
import pandas as pd

In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import itertools

In [2]:
#chunk data by statements to avoid RAM overflow
print("Load data")
all_data = pd.read_csv("/Users/abhijit/IGRI Files/repo/step1_edit_data_sample.csv")
statements = all_data.groupby('sid')
sid = all_data['sid'].unique()
print(sid)
all_features = []

Load data
[328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345
 347 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365
 366 367 368 369 370 371 372 383 384 385 386 387 388 389 390 391 392 393
 394 395 396 397 398 399 400 401 402 403 404 405 406 407 408 414 415 416
 417 418 419 420 421 422 423 424 425 426 427 428 429 431 432 433 434 435
 436 437 439 441 442 443 444 445 447 448 449 450 451 452 453 454 455 456
 457 458 459 460 461 462 464 465 466 467 468 469 470 471 472 473 474 475
 476 477 478 479 480 481 482 483 484 485 486 487 488 489 490 491 492 493
 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509 510 511
 512 513 514 515 518 519 520 521 522 523 524 525 526 528 529 531 533 534
 536 538 539 541 543 545 547 549 550 552 553 555 556 557 558 559 560 561
 562 563 564 565 566 567 568 569 570 571 572 573 574 575 576 577 578 579
 584 585 586 587 588 589 590 591 592 593 594 595 596 597 598 599 601 602
 603 604 605 606 607 608 609 610 611 612 

In [3]:
# Pretrained model
print("Load model")
# For DistilBERT:
print( "Model: distilbert-base-uncased")
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

Load model
Model: distilbert-base-uncased


In [4]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [5]:
for statement_no in sid:
  print(statement_no)
  df = statements.get_group(statement_no)
  df = df.rename(columns=dict(zip(df.columns,[0, "sid", "sid_text","tid",1,2])))
  #print(df.shape)
  #batch_1 = df[:2000].reset_index()
  ### clean up sentences to preempt tokenization problems
  ### this is all effort to make the space tokenization look more like the bert tokenization, especially special handling of ellipses and hypthens
  print(df.head(5))
  df = df.assign(sid_text=df['sid_text'].apply(
      lambda x:x.lower().replace('-lrb-', '[').replace('-rrb-', ']').replace('...', '<ELLIPSES>').replace('.', '').replace( '<ELLIPSES>', '...').replace(' - ', ' , ').replace('-', '').replace('  ', ' ').strip()
      ))
  #df = df.assign(sid_text_old=df['sid_text'])
  #df = df.assign(sid_text=df['sid_text'].apply(
      #lambda x:x.lower().replace('.', '').replace('-', '').replace('  ', ' ').replace('root ', '').strip()
      #))

  batch_1 = df
  batch_1[1].value_counts()

  print("Prep data")
  # Tokenization
  tokenized_raw = batch_1['sid_text'].apply((lambda x: tokenizer.tokenize(x)))
  tokenized = batch_1['sid_text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  tokens_ = tokenized_raw.tolist()[0]

  print(tokenized_raw)
  print(tokenized)
  print(tokens_)

  # Padding
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
      max_len = len(i)
      
  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  print("  check padding: ", np.array(padded).shape)

  # Masking
  attention_mask = np.where(padded != 0, 1, 0)
  print("  check mask:", attention_mask.shape)

  bertids = []
  # Implementing/Checking sentence merge
  df = df.assign(bertid=-1) 

  for s, _ in df.iterrows():
    dep_feats = df.loc[s].values.tolist() # labeled token and its metadata

    print('DEP FEATS ' + str(dep_feats))
    word = dep_feats[0].replace('-','').replace('.', '').replace('LRB', '[').replace('RRB', ']').lower() #.partition('/')[0]
    
    tok_index = np.array([True if word.startswith(elem) else False for x,elem in enumerate(tokens_)])
    bertid = np.argwhere(tok_index).ravel().tolist()
    print("TOKEN INDEX " + str(tok_index))
    print("BERTID " + str(bertid))
    
    if len(bertids):
      bertids_ = []
      #greater than minimum possible indices of previous pos
      print(word,bertids,bertid)
      bertid = list(filter(lambda x: x > max([min(elem) for elem in (bertids)]),bertid))
      for index,elem in enumerate(bertids): 
        #less than maximum possible indices of next pos
        # max_ = min(min(elem),max(bertid)) #min(list(itertools.chain.from_iterable(bertids[index:])))
        bertids_.append(list(filter(lambda x: x < max(bertid),elem)))
      bertids = bertids_.copy()

    #   bertids.append(temp)
    # else:
    bertids.append(bertid)

  bertids = [x[0] for x in bertids]; bertids.sort()
  print("BERTIDS 1 " + str(bertids))
  # combs = list(itertools.product(*bertids))
  # bertids = []
  # for order in combs:
  #   order = list(order)
  #   if len(tokenized_raw) == len(set(order)): bertids = order

  bertids.sort(); print("BERTIDS 2 " + str(bertids))
  df["bertid"] = bertids
  print("Get embeddings (this is the slow part seems)")
  # Querying the model
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  #print(df)
  with torch.no_grad():
    last_hidden_states, hidden_states = model(input_ids, attention_mask=attention_mask, output_hidden_states=True)

  nwords = hidden_states[0].shape[0]
  nsentidxs = hidden_states[0].shape[1]
  nfeats = hidden_states[0].shape[2]
    
  print("NWORDS " + str(nwords))
  print("NSENT ID " + str(nsentidxs))
  print("NFEATS " + str(nfeats))
  #print("SHAPE " + np.array(hidden_states).shape)
  print("MAX LENGTH " + str(max_len))
  ### save costly output
  torch.save(hidden_states, "/Users/abhijit/IGRI Files/repo/step25_hidden_states.pt")
  #torch.load("data/step25_hidden_states20201129rice_orig_filtering.pt")
  #torch.load("data/step25_hidden_states_20201129frey_less_filtering.pt)
  #torch.load("data/step25_hidden_states.pt")

  if False: # get last layer
    features = last_hidden_states[:,1,:].numpy()
  if False: # get last layer, by word
    mask = torch.tensor([[[v,]*nfeats,] for v in df.bertid])
    features = torch.gather(hidden_states[6][:,:,:], 1, mask)
    ### get to right dimensionality and type
    features  = features[:,0,:].numpy()
    ### tested picking out with this line
    ###  torch.gather(hidden_states[6][::50,:,0], 1, torch.tensor([[v%169,] for v in range(172)]))[-10:]
  if True: # get word embeddings for last four layers plus whole sentence embedding, last layer
    mask = torch.tensor([[[v,]*nfeats,] for v in df.bertid])
    features = np.concatenate((
                              hidden_states[6][:,0,:].numpy(),
                              torch.gather(hidden_states[6][:,:,:], 1, mask)[:,0,:].numpy(),
                              torch.gather(hidden_states[5][:,:,:], 1, mask)[:,0,:].numpy(),
                              torch.gather(hidden_states[4][:,:,:], 1, mask)[:,0,:].numpy(),
                              torch.gather(hidden_states[3][:,:,:], 1, mask)[:,0,:].numpy()
                            ),
                            axis=1)


  # sanity check model output format
  assert( np.all(hidden_states[-1].numpy() == last_hidden_states[:,:,:].numpy()) )
  labels = batch_1[1]
  all_features.append(features)

328
           0  sid                                           sid_text  tid  \
0        and  328  the voting members of the board shall be compr...   12   
1  residents  328  the voting members of the board shall be compr...   11   
2         of  328  the voting members of the board shall be compr...   10   
3      board  328  the voting members of the board shall be compr...    6   
4         of  328  the voting members of the board shall be compr...    4   

           1  2  
0     Object  4  
1     Object  4  
2        Aim  0  
3  Attribute  1  
4  Attribute  1  
Prep data
0     [the, voting, members, of, the, board, shall, ...
1     [the, voting, members, of, the, board, shall, ...
2     [the, voting, members, of, the, board, shall, ...
3     [the, voting, members, of, the, board, shall, ...
4     [the, voting, members, of, the, board, shall, ...
5     [the, voting, members, of, the, board, shall, ...
6     [the, voting, members, of, the, board, shall, ...
7     [the, voting, mem

ValueError: max() arg is an empty sequence

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

In [ ]:
(hidden_states[0]).shape

In [ ]:
len((all_features)[420][4])
#len(all_data['sid'].unique())

In [ ]:
(all_features[0])

In [ ]:
all_features = np.concatenate(all_features,axis=0);print("Embeds produced: ",all_features.shape)
np.savetxt("/Users/abhijit/IGRI Files/repo/step1_edit_embedding_features.csv", all_features, delimiter=",", header=",".join(map(str,range(features.shape[1]))))
